In [2]:
from Flamingo.utils.pretty import pretty_print, print_local_vars, vis_model
from Flamingo.utils.test_model import MyClassifier
from loguru import logger

In [3]:
model = MyClassifier(3, 100, ch_multi=128)

In [5]:
x = vis_model(model)

root
├── stage1 (Sequential)
│   ├── 0 (Sequential)
│   │   ├── 0 (Conv2d) weight:[128, 3, 2, 2] bias:[128]
│   │   └── 1 (GroupNorm) weight:[128] bias:[128]
│   └── 1-2(ConvBlock) layer_scale:[]
│       └── residual (Sequential)
│           ├── 0-0,3-3(GroupNorm) weight:[128] bias:[128]
│           └── 2-2,5-5(Conv2d) weight:[128, 128, 3, 3] bias:[128]
├── stage2 (Sequential)
│   ├── 0 (Sequential)
│   │   ├── 0 (Conv2d) weight:[256, 128, 2, 2] bias:[256]
│   │   └── 1 (GroupNorm) weight:[256] bias:[256]
│   └── 1-2(ConvBlock) layer_scale:[]
│       └── residual (Sequential)
│           ├── 0-0,3-3(GroupNorm) weight:[256] bias:[256]
│           └── 2-2,5-5(Conv2d) weight:[256, 256, 3, 3] bias:[256]
├── stage3 (Sequential)
│   ├── 0 (Sequential)
│   │   ├── 0 (Conv2d) weight:[512, 256, 2, 2] bias:[512]
│   │   └── 1 (GroupNorm) weight:[512] bias:[512]
│   └── 1-2(ConvBlock) layer_scale:[]
│       └── residual (Sequential)
│           ├── 0-0,3-3(GroupNorm) weight:[512] bias:[512]
│           └── 2-2,5-5(Conv2d) weight:[512, 512, 3, 3] bias:[512]
├── stage4 (Sequential)
│   ├── 0 (Sequential)
│   │   ├── 0 (Conv2d) weight:[1024, 512, 2, 2] bias:[1024]
│   │   └── 1 (GroupNorm) weight:[1024] bias:[1024]
│   └── 1-2(ConvBlock) layer_scale:[]
│       └── residual (Sequential)
│           ├── 0-0,3-3(GroupNorm) weight:[1024] bias:[1024]
│           └── 2-2,5-5(Conv2d) weight:[1024, 1024, 3, 3] bias:[1024]
└── classifier (Sequential)
    └── 2 (Linear) weight:[100, 1024] bias:[100]

In [26]:
logger.info(model)

2023-12-17 15:04:19.109 | INFO     | __main__:<module>:1 - MyClassifier(
  (stage1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(2, 2), stride=(2, 2))
      (1): GroupNorm(1, 128, eps=1e-05, affine=True)
    )
    (1): ConvBlock(
      (residual): Sequential(
        (0): GroupNorm(1, 128, eps=1e-05, affine=True)
        (1): GELU(approximate='none')
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): GroupNorm(1, 128, eps=1e-05, affine=True)
        (4): GELU(approximate='none')
        (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): ConvBlock(
      (residual): Sequential(
        (0): GroupNorm(1, 128, eps=1e-05, affine=True)
        (1): GELU(approximate='none')
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): GroupNorm(1, 128, eps=1e-05, affine=True)
        (4): GELU(approximate='none')
        (5): Conv2d(128, 128, ker

In [25]:

# print(x)

stage1 (Sequential)
├── 0 (Sequential)
│   ├── 0 (Conv2d) weight:[128, 3, 2, 2] bias:[128]
│   └── 1 (GroupNorm) weight:[128] bias:[128]
└── 1-2(ConvBlock) layer_scale:[]
    └── residual (Sequential)
        ├── 0-0,3-3(GroupNorm) weight:[128] bias:[128]
        └── 2-2,5-5(Conv2d) weight:[128, 128, 3, 3] bias:[128]

a decorator to print local varibles in function scope

In [4]:
a = dict(
    a=0,
    b="hello",
    c=[1,2,3]
)
b = "hellow world"

In [5]:
@print_local_vars
def function_scope(a, b):
    return a['b'] + "   " + b
function_scope(a, b)

Local ! 🧛
{
    'args': ({'a': 0, 'b': 'hello', 'c': [1, 2, 3]}, 'hellow world'),
    'kwargs': {},
    'result': 'hello   hellow world',
    'func': <function function_scope at 0x7fdde7a2de50>
}

'hello   hellow world'

In [1]:
from loguru import logger

logger.debug("That's it, beautiful and simple logging!")

2023-12-16 22:48:33.612 | DEBUG    | __main__:<module>:3 - That's it, beautiful and simple logging!


In [2]:
logger.add("file_{time}.log")

1

In [8]:
work_dir = "logg"
logger.add(work_dir + "/{time}.log", enqueue=True)
# logger.add(sink="logg/{}file_{}.log".format("test"), enqueue=True)

4

In [3]:
logger.debug("That's it, beautiful and simple logging!")

2023-12-16 22:52:37.650 | DEBUG    | __main__:<module>:1 - That's it, beautiful and simple logging!


In [4]:
logger.info("Hello world")

2023-12-16 22:57:55.204 | INFO     | __main__:<module>:1 - Hello world
